# Introduction to Linear Algebra with NumPy and PyTorch
by Pierre Nugues


## The Corpus
We create a dictionary with URLs

In [1]:
classics_url = {'iliad': 'http://classics.mit.edu/Homer/iliad.mb.txt',
                'odyssey': 'http://classics.mit.edu/Homer/odyssey.mb.txt',
                'eclogue': 'http://classics.mit.edu/Virgil/eclogue.mb.txt',
                'georgics': 'http://classics.mit.edu/Virgil/georgics.mb.txt',
                'aeneid': 'http://classics.mit.edu/Virgil/aeneid.mb.txt'}


We read the texts from the URLs

In [2]:
import requests

classics = {}
for key in classics_url:
    classics[key] = requests.get(classics_url[key]).text


We remove the license information to keep only the text

In [4]:
text_bounds = {'iliad': (136, -486), 'odyssey': (138, -486),
               'eclogue': (139, -486), 'georgics': (140, -486), 'aeneid': (138, -486)}


In [5]:
for key in classics:
    classics[key] = classics[key][text_bounds[key][0]:text_bounds[key][1]]


In [8]:
classics['iliad'][:50]


'The Iliad\nBy Homer\n\n\nTranslated by Samuel Butler\n\n'

We additionally write the Iliad and the Odyssey in two text files

In [9]:
with open('iliad.txt', 'w') as f_il, open('odyssey.txt', 'w') as f_od:
    f_il.write(classics['iliad'])
    f_od.write(classics['odyssey'])


We store the corpus in a JSON file

In [10]:
import json

with open('classics.json', 'w') as f:
    json.dump(classics, f)


We read it again

In [11]:
with open('classics.json', 'r') as f:
    classics = json.loads(f.read())


## Utilities

In [12]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'


In [13]:
class Text:
    """Text class to hold and process text"""

    alphabet = 'abcdefghijklmnopqrstuvwxyz'

    def __init__(self, text=None):
        """The constructor called when an object
        is created"""

        self.content = text
        self.length = len(text)
        self.letter_counts = {}

    def count_letters(self, lc=True):
        """Function to count the letters of a text"""

        letter_counts = {}
        if lc:
            text = self.content.lower()
        else:
            text = self.content
        for letter in text:
            if letter.lower() in self.alphabet:
                if letter in letter_counts:
                    letter_counts[letter] += 1
                else:
                    letter_counts[letter] = 1
        self.letter_counts = letter_counts
        return letter_counts


In [88]:
a = r' This is a test \x40'
a

' This is a test \\x40'

## Imports

In [13]:
import math
import random
import numpy as np
import torch


In [14]:
random.seed(4321)
np.random.seed(4321)
torch.manual_seed(4321)


## The Dataset

Let us read Homer's _Iliad_ and _Odyssey_ and Virgil's _Eclogue_, _Georgics_, and _Aeneid_.

In [15]:
titles = ['iliad', 'odyssey', 'eclogue', 'georgics', 'aeneid']
titles


['iliad', 'odyssey', 'eclogue', 'georgics', 'aeneid']

In [16]:
texts = []
for title in titles:
    texts += [classics[title]]


In [17]:
cnt_dicts = []
for text in texts:
    cnt_dicts += [Text(text).count_letters()]


In [18]:
cnt_lists = []
for cnt_dict in cnt_dicts:
    cnt_lists += [list(map(cnt_dict.get, alphabet))]


In [19]:
cnt_lists[0][:3]


[51020, 8941, 11558]

In [20]:
for i, cnt_list in enumerate(cnt_lists):
    print(titles[i], cnt_lists[i][:10])


iliad [51020, 8941, 11558, 28333, 77466, 16114, 12595, 50194, 38151, 1624]
odyssey [37630, 6598, 8580, 20738, 59783, 10449, 9803, 34787, 28793, 424]
eclogue [2716, 578, 723, 1440, 4366, 846, 808, 2509, 2252, 22]
georgics [6841, 1619, 2017, 4027, 12112, 2424, 2150, 6988, 6038, 59]
aeneid [36678, 6869, 10023, 23866, 55372, 11618, 9607, 33057, 30579, 908]


## Vectors
### NumPy

In [21]:
np.array([2, 3])
np.array([1, 2, 3])


array([1, 2, 3])

Vectors of letter counts

In [22]:
iliad_cnt = np.array(cnt_lists[0])
odyssey_cnt = np.array(cnt_lists[1])
eclogue_cnt = np.array(cnt_lists[2])
georgics_cnt = np.array(cnt_lists[3])
aeneid_cnt = np.array(cnt_lists[4])


In [23]:
iliad_cnt


array([51020,  8941, 11558, 28333, 77466, 16114, 12595, 50194, 38151,
        1624,  4413, 25311, 16648, 42194, 51270,  9104,   283, 36457,
       41243, 54177, 18409,  6060, 15665,   597, 11908,   284])

In [24]:
odyssey_cnt


array([37630,  6598,  8580, 20738, 59783, 10449,  9803, 34787, 28793,
         424,  3631, 18951, 13060, 31889, 38778,  6679,   256, 25668,
       31352, 40483, 15406,  4803, 12989,   350, 10974,   124])

### The datatype

In [25]:
odyssey_cnt.dtype


dtype('int64')

In [26]:
vector = np.array([1, 2, 3], dtype='int32')
vector


array([1, 2, 3], dtype=int32)

In [27]:
vector.dtype


dtype('int32')

In [28]:
vector = np.array([1, 2, 3], dtype='float64')
vector


array([1., 2., 3.])

In [29]:
np.array([0, 1, 2, 3], dtype='bool')


array([False,  True,  True,  True])

### The vector size

In [30]:
odyssey_cnt.shape


(26,)

### Indices and Slices

In [31]:
vector = np.array([1, 2, 3, 4])
vector[1]   # 2
vector[:1]  # array([1])
vector[1:3]  # array([2, 3])


array([2, 3])

### Operations

In [32]:
np.array([1, 2, 3]) + np.array([4, 5, 6])


array([5, 7, 9])

In [33]:
3 * np.array([1, 2, 3])


array([3, 6, 9])

In [34]:
iliad_cnt + odyssey_cnt      # array([88650,  15539,  20138, ...])


array([ 88650,  15539,  20138,  49071, 137249,  26563,  22398,  84981,
        66944,   2048,   8044,  44262,  29708,  74083,  90048,  15783,
          539,  62125,  72595,  94660,  33815,  10863,  28654,    947,
        22882,    408])

In [35]:
iliad_cnt - odyssey_cnt      # array([13390,  2343,  2978, ...])


array([13390,  2343,  2978,  7595, 17683,  5665,  2792, 15407,  9358,
        1200,   782,  6360,  3588, 10305, 12492,  2425,    27, 10789,
        9891, 13694,  3003,  1257,  2676,   247,   934,   160])

In [36]:

iliad_cnt - 2 * odyssey_cnt  # array([-24240,  -4255,  ...])


array([-24240,  -4255,  -5602, -13143, -42100,  -4784,  -7011, -19380,
       -19435,    776,  -2849, -12591,  -9472, -21584, -26286,  -4254,
         -229, -14879, -21461, -26789, -12403,  -3546, -10313,   -103,
       -10040,     36])

### Comparison with lists

In [37]:
[1, 2, 3] + [4, 5, 6]


[1, 2, 3, 4, 5, 6]

In [38]:
3 * [1, 2, 3]


[1, 2, 3, 1, 2, 3, 1, 2, 3]

### PyTorch
#### Tensors

In [39]:
torch.tensor([2, 3])
torch.tensor([1, 2, 3])


tensor([1, 2, 3])

In [40]:
iliad_cnt_pt = torch.tensor(cnt_lists[0])


In [41]:
iliad_cnt_pt


tensor([51020,  8941, 11558, 28333, 77466, 16114, 12595, 50194, 38151,  1624,
         4413, 25311, 16648, 42194, 51270,  9104,   283, 36457, 41243, 54177,
        18409,  6060, 15665,   597, 11908,   284])

#### Types

In [42]:
torch.tensor([1, 2, 3]).dtype


torch.int64

In [43]:
torch.tensor([1, 2, 3], dtype=torch.float64)


tensor([1., 2., 3.], dtype=torch.float64)

In [44]:
torch.tensor([0, 1, 2, 3], dtype=torch.bool)


tensor([False,  True,  True,  True])

#### Size

In [45]:
iliad_cnt_pt.size()


torch.Size([26])

#### NumPy/PyTorch Conversion

In [46]:
np_array = np.array([1, 2, 3])
tensor = torch.from_numpy(np_array)
tensor


tensor([1, 2, 3])

In [47]:
tensor = torch.tensor([1, 2, 3])
np_array = tensor.numpy()
np_array


array([1, 2, 3])

#### Device

In [48]:
torch.cuda.is_available()


False

In [49]:
torch.backends.mps.is_available()


False

In [50]:
torch.device('cpu')


device(type='cpu')

In [51]:
torch.device('mps')


device(type='mps')

In [52]:
tensor = torch.tensor([1, 2, 3])
tensor.device


device(type='cpu')

In [53]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')


In [54]:
device


device(type='cpu')

In [55]:
tensor = torch.tensor([1, 2, 3], device=device)
tensor


tensor([1, 2, 3])

In [56]:
tensor = torch.tensor([1, 2, 3])
tensor.to(device)


tensor([1, 2, 3])

## NumPy Functions

In [57]:
np.set_printoptions(precision=3)


In [58]:
np.sqrt(iliad_cnt)


array([225.876,  94.557, 107.508, 168.324, 278.327, 126.941, 112.227,
       224.04 , 195.323,  40.299,  66.43 , 159.094, 129.027, 205.412,
       226.429,  95.415,  16.823, 190.937, 203.084, 232.76 , 135.68 ,
        77.846, 125.16 ,  24.434, 109.124,  16.852])

In [59]:
np.cos(iliad_cnt)


array([ 0.86 ,  1.   , -0.997, -0.52 ,  0.821, -0.717, -0.938, -0.715,
        0.877, -0.979, -0.592, -0.688, -0.765, -0.745,  0.702,  0.944,
        0.967, -0.378,  0.985, -0.973,  0.743, -0.991,  0.524,  0.995,
        0.205,  0.309])

In [60]:
math.sqrt(iliad_cnt)


TypeError: only length-1 arrays can be converted to Python scalars

In [61]:
np_sqrt = np.vectorize(math.sqrt)
np_sqrt(iliad_cnt)


array([225.876,  94.557, 107.508, 168.324, 278.327, 126.941, 112.227,
       224.04 , 195.323,  40.299,  66.43 , 159.094, 129.027, 205.412,
       226.429,  95.415,  16.823, 190.937, 203.084, 232.76 , 135.68 ,
        77.846, 125.16 ,  24.434, 109.124,  16.852])

In [62]:
np.sum(odyssey_cnt)


472978

In [63]:
iliad_dist = iliad_cnt / np.sum(iliad_cnt)
odyssey_dist = odyssey_cnt / np.sum(odyssey_cnt)


In [64]:
iliad_dist


array([0.081, 0.014, 0.018, 0.045, 0.123, 0.026, 0.02 , 0.08 , 0.061,
       0.003, 0.007, 0.04 , 0.026, 0.067, 0.081, 0.014, 0.   , 0.058,
       0.065, 0.086, 0.029, 0.01 , 0.025, 0.001, 0.019, 0.   ])

In [65]:
odyssey_dist


array([0.08 , 0.014, 0.018, 0.044, 0.126, 0.022, 0.021, 0.074, 0.061,
       0.001, 0.008, 0.04 , 0.028, 0.067, 0.082, 0.014, 0.001, 0.054,
       0.066, 0.086, 0.033, 0.01 , 0.027, 0.001, 0.023, 0.   ])

PyTorch

In [66]:
torch.sqrt(iliad_cnt_pt)


tensor([225.8761,  94.5569, 107.5081, 168.3241, 278.3271, 126.9409, 112.2274,
        224.0402, 195.3228,  40.2989,  66.4304, 159.0943, 129.0271, 205.4118,
        226.4288,  95.4149,  16.8226, 190.9372, 203.0837, 232.7595, 135.6798,
         77.8460, 125.1599,  24.4336, 109.1238,  16.8523])

In [67]:
torch.sum(iliad_cnt_pt)


tensor(630019)

## Dot Product


In [68]:
np.dot(iliad_dist, odyssey_dist)


0.06581109734214702

In [69]:
iliad_dist @ odyssey_dist


0.06581109734214702

In [70]:
torch.dot(torch.tensor(iliad_dist), torch.tensor(odyssey_dist))


tensor(0.0658, dtype=torch.float64)

### Norm

In [71]:
np.linalg.norm([1.0, 2.0, 3.0])


3.7416573867739413

In [72]:
torch.norm(torch.tensor([1.0, 2.0, 3.0]))


tensor(3.7417)

#### Cosine

Finally, we compute the cosine 
$$
\frac{\mathbf{x} \cdot \mathbf{y}}{||\mathbf{x}|| . ||\mathbf{y}||}.
$$

In [73]:
(iliad_dist @ odyssey_dist) / (
    np.linalg.norm(iliad_dist) *
    np.linalg.norm(odyssey_dist))


0.9990782943375434

## Matrices

### NumPy

We create a matrix from the list of lists

In [74]:
hv_cnts = np.array(cnt_lists)
hv_cnts


array([[51020,  8941, 11558, 28333, 77466, 16114, 12595, 50194, 38151,
         1624,  4413, 25311, 16648, 42194, 51270,  9104,   283, 36457,
        41243, 54177, 18409,  6060, 15665,   597, 11908,   284],
       [37630,  6598,  8580, 20738, 59783, 10449,  9803, 34787, 28793,
          424,  3631, 18951, 13060, 31889, 38778,  6679,   256, 25668,
        31352, 40483, 15406,  4803, 12989,   350, 10974,   124],
       [ 2716,   578,   723,  1440,  4366,   846,   808,  2509,  2252,
           22,   268,  1809,  1043,  2248,  2948,   569,    12,  2236,
         2618,  2940,  1031,   361,  1023,    38,   906,    22],
       [ 6841,  1619,  2017,  4027, 12112,  2424,  2150,  6988,  6038,
           59,   782,  4309,  2027,  6552,  6958,  1669,    53,  6704,
         7143,  8713,  2583,   903,  2480,    85,  1458,    64],
       [36678,  6869, 10023, 23866, 55372, 11618,  9607, 33057, 30579,
          908,  2702, 18768, 10201, 32258, 32595,  8343,   530, 32077,
        36430, 39481, 13714,  

The size

In [75]:
hv_cnts.shape


(5, 26)

The data type

In [76]:
hv_cnts.dtype


dtype('int64')

### Indices and Slices

In [77]:
iliad_cnt[2]


11558

In [78]:
hv_cnts[1, 2]


8580

Slices

In [79]:
hv_cnts[1, :]


array([37630,  6598,  8580, 20738, 59783, 10449,  9803, 34787, 28793,
         424,  3631, 18951, 13060, 31889, 38778,  6679,   256, 25668,
       31352, 40483, 15406,  4803, 12989,   350, 10974,   124])

In [80]:
hv_cnts[1, :2]


array([37630,  6598])

In [81]:
hv_cnts[3, 2:4]


array([2017, 4027])

In [82]:
hv_cnts[3:, 2:4]


array([[ 2017,  4027],
       [10023, 23866]])

In [83]:
hv_cnts[:, 2]


array([11558,  8580,   723,  2017, 10023])

### Number of indices

In [84]:
odyssey_cnt.ndim


1

In [85]:
hv_cnts.ndim


2

## Addition and multiplication by a scalar

In [86]:
hv_cnts - 2 * hv_cnts


array([[-51020,  -8941, -11558, -28333, -77466, -16114, -12595, -50194,
        -38151,  -1624,  -4413, -25311, -16648, -42194, -51270,  -9104,
          -283, -36457, -41243, -54177, -18409,  -6060, -15665,   -597,
        -11908,   -284],
       [-37630,  -6598,  -8580, -20738, -59783, -10449,  -9803, -34787,
        -28793,   -424,  -3631, -18951, -13060, -31889, -38778,  -6679,
          -256, -25668, -31352, -40483, -15406,  -4803, -12989,   -350,
        -10974,   -124],
       [ -2716,   -578,   -723,  -1440,  -4366,   -846,   -808,  -2509,
         -2252,    -22,   -268,  -1809,  -1043,  -2248,  -2948,   -569,
           -12,  -2236,  -2618,  -2940,  -1031,   -361,  -1023,    -38,
          -906,    -22],
       [ -6841,  -1619,  -2017,  -4027, -12112,  -2424,  -2150,  -6988,
         -6038,    -59,   -782,  -4309,  -2027,  -6552,  -6958,  -1669,
           -53,  -6704,  -7143,  -8713,  -2583,   -903,  -2480,    -85,
         -1458,    -64],
       [-36678,  -6869, -10023, -238

### PyTorch

In [87]:
hv_cnts_pt = torch.tensor(cnt_lists)
hv_cnts_pt


tensor([[51020,  8941, 11558, 28333, 77466, 16114, 12595, 50194, 38151,  1624,
          4413, 25311, 16648, 42194, 51270,  9104,   283, 36457, 41243, 54177,
         18409,  6060, 15665,   597, 11908,   284],
        [37630,  6598,  8580, 20738, 59783, 10449,  9803, 34787, 28793,   424,
          3631, 18951, 13060, 31889, 38778,  6679,   256, 25668, 31352, 40483,
         15406,  4803, 12989,   350, 10974,   124],
        [ 2716,   578,   723,  1440,  4366,   846,   808,  2509,  2252,    22,
           268,  1809,  1043,  2248,  2948,   569,    12,  2236,  2618,  2940,
          1031,   361,  1023,    38,   906,    22],
        [ 6841,  1619,  2017,  4027, 12112,  2424,  2150,  6988,  6038,    59,
           782,  4309,  2027,  6552,  6958,  1669,    53,  6704,  7143,  8713,
          2583,   903,  2480,    85,  1458,    64],
        [36678,  6869, 10023, 23866, 55372, 11618,  9607, 33057, 30579,   908,
          2702, 18768, 10201, 32258, 32595,  8343,   530, 32077, 36430, 39481,
  

In [88]:
hv_cnts_pt.dtype


torch.int64

In [89]:
hv_cnts_pt.size()


torch.Size([5, 26])

### NumPy Functions

In [90]:
np.set_printoptions(precision=3)


In [91]:
np.cos(hv_cnts)


array([[ 0.86 ,  1.   , -0.997, -0.52 ,  0.821, -0.717, -0.938, -0.715,
         0.877, -0.979, -0.592, -0.688, -0.765, -0.745,  0.702,  0.944,
         0.967, -0.378,  0.985, -0.973,  0.743, -0.991,  0.524,  0.995,
         0.205,  0.309],
       [ 1.   ,  0.793, -0.952, -0.94 ,  0.063,  0.998,  0.333, -0.99 ,
        -0.954, -0.993,  0.777,  0.611, -0.921, -0.261, -0.246,  1.   ,
        -0.04 ,  0.373,  0.458,  0.906,  0.932, -0.88 , -0.085, -0.284,
        -0.914, -0.093],
       [-0.093,  0.999,  0.907,  0.408,  0.687, -0.613, -0.819, -0.424,
        -0.867, -1.   , -0.57 ,  0.849,  1.   ,  0.189,  0.375, -0.932,
         0.844,  0.687, -0.495,  0.862,  0.849, -0.96 ,  0.4  ,  0.955,
         0.342, -1.   ],
       [ 0.181, -0.472,  0.995,  0.867, -0.399,  0.258,  0.408,  0.455,
         0.99 , -0.771, -0.967,  0.301, -0.782,  0.207, -0.809, -0.686,
        -0.918,  0.987,  0.556, -0.206,  0.819, -0.206, -0.283, -0.984,
         0.955,  0.392],
       [-0.996,  0.092,  0.249, -0.7

In [92]:
np.sum(hv_cnts)


1706121

In [93]:
np.sum(hv_cnts, axis=0)


array([134885,  24605,  32901,  78404, 209099,  41451,  34963, 127535,
       105813,   3037,  11796,  69148,  42979, 115141, 132549,  26364,
         1134, 103142, 118786, 145794,  51143,  16502,  43254,   1630,
        33302,    764])

In [94]:
np.sum(hv_cnts, axis=1)


array([630019, 472978,  36332,  96758, 470034])

### PyTorch

In [95]:
torch.sum(hv_cnts_pt)


tensor(1706121)

In [96]:
torch.sum(hv_cnts_pt, dim=0)  # array([ 134885,  24605,  32901, ...])


tensor([134885,  24605,  32901,  78404, 209099,  41451,  34963, 127535, 105813,
          3037,  11796,  69148,  42979, 115141, 132549,  26364,   1134, 103142,
        118786, 145794,  51143,  16502,  43254,   1630,  33302,    764])

In [97]:
torch.sum(hv_cnts_pt, dim=1)  # array([630019, 472978,  36332,  96758, 470034])


tensor([630019, 472978,  36332,  96758, 470034])

### Transposing and Reshaping Arrays

In [98]:
hv_cnts.T


array([[51020, 37630,  2716,  6841, 36678],
       [ 8941,  6598,   578,  1619,  6869],
       [11558,  8580,   723,  2017, 10023],
       [28333, 20738,  1440,  4027, 23866],
       [77466, 59783,  4366, 12112, 55372],
       [16114, 10449,   846,  2424, 11618],
       [12595,  9803,   808,  2150,  9607],
       [50194, 34787,  2509,  6988, 33057],
       [38151, 28793,  2252,  6038, 30579],
       [ 1624,   424,    22,    59,   908],
       [ 4413,  3631,   268,   782,  2702],
       [25311, 18951,  1809,  4309, 18768],
       [16648, 13060,  1043,  2027, 10201],
       [42194, 31889,  2248,  6552, 32258],
       [51270, 38778,  2948,  6958, 32595],
       [ 9104,  6679,   569,  1669,  8343],
       [  283,   256,    12,    53,   530],
       [36457, 25668,  2236,  6704, 32077],
       [41243, 31352,  2618,  7143, 36430],
       [54177, 40483,  2940,  8713, 39481],
       [18409, 15406,  1031,  2583, 13714],
       [ 6060,  4803,   361,   903,  4375],
       [15665, 12989,  1023,  24

In [99]:
iliad_cnt.T


array([51020,  8941, 11558, 28333, 77466, 16114, 12595, 50194, 38151,
        1624,  4413, 25311, 16648, 42194, 51270,  9104,   283, 36457,
       41243, 54177, 18409,  6060, 15665,   597, 11908,   284])

In [100]:
np.array([iliad_cnt])


array([[51020,  8941, 11558, 28333, 77466, 16114, 12595, 50194, 38151,
         1624,  4413, 25311, 16648, 42194, 51270,  9104,   283, 36457,
        41243, 54177, 18409,  6060, 15665,   597, 11908,   284]])

In [101]:
np.array([iliad_cnt]).shape


(1, 26)

In [102]:
np.array([iliad_cnt]).T


array([[51020],
       [ 8941],
       [11558],
       [28333],
       [77466],
       [16114],
       [12595],
       [50194],
       [38151],
       [ 1624],
       [ 4413],
       [25311],
       [16648],
       [42194],
       [51270],
       [ 9104],
       [  283],
       [36457],
       [41243],
       [54177],
       [18409],
       [ 6060],
       [15665],
       [  597],
       [11908],
       [  284]])

In [103]:
iliad_cnt.reshape(1, -1)


array([[51020,  8941, 11558, 28333, 77466, 16114, 12595, 50194, 38151,
         1624,  4413, 25311, 16648, 42194, 51270,  9104,   283, 36457,
        41243, 54177, 18409,  6060, 15665,   597, 11908,   284]])

In [104]:
iliad_cnt.reshape(-1, 1)


array([[51020],
       [ 8941],
       [11558],
       [28333],
       [77466],
       [16114],
       [12595],
       [50194],
       [38151],
       [ 1624],
       [ 4413],
       [25311],
       [16648],
       [42194],
       [51270],
       [ 9104],
       [  283],
       [36457],
       [41243],
       [54177],
       [18409],
       [ 6060],
       [15665],
       [  597],
       [11908],
       [  284]])

In [105]:
torch.unsqueeze(torch.tensor([1, 2, 3]), 1)


tensor([[1],
        [2],
        [3]])

### Broadcasting

Relative frequencies of the letter counts

In [106]:
iliad_dist = (1/np.sum(iliad_cnt)) * iliad_cnt
odyssey_dist = (1/np.sum(odyssey_cnt)) * odyssey_cnt


In [107]:
iliad_cnt / np.sum(iliad_cnt)


array([0.081, 0.014, 0.018, 0.045, 0.123, 0.026, 0.02 , 0.08 , 0.061,
       0.003, 0.007, 0.04 , 0.026, 0.067, 0.081, 0.014, 0.   , 0.058,
       0.065, 0.086, 0.029, 0.01 , 0.025, 0.001, 0.019, 0.   ])

In [108]:
odyssey_cnt / np.sum(odyssey_cnt)


array([0.08 , 0.014, 0.018, 0.044, 0.126, 0.022, 0.021, 0.074, 0.061,
       0.001, 0.008, 0.04 , 0.028, 0.067, 0.082, 0.014, 0.001, 0.054,
       0.066, 0.086, 0.033, 0.01 , 0.027, 0.001, 0.023, 0.   ])

We can apply an elementwise multiplication or division

In [109]:
np.array([np.sum(hv_cnts, axis=1)]).T


array([[630019],
       [472978],
       [ 36332],
       [ 96758],
       [470034]])

In [110]:
hv_dist = hv_cnts / np.array([np.sum(hv_cnts, axis=1)]).T
hv_dist


array([[0.081, 0.014, 0.018, 0.045, 0.123, 0.026, 0.02 , 0.08 , 0.061,
        0.003, 0.007, 0.04 , 0.026, 0.067, 0.081, 0.014, 0.   , 0.058,
        0.065, 0.086, 0.029, 0.01 , 0.025, 0.001, 0.019, 0.   ],
       [0.08 , 0.014, 0.018, 0.044, 0.126, 0.022, 0.021, 0.074, 0.061,
        0.001, 0.008, 0.04 , 0.028, 0.067, 0.082, 0.014, 0.001, 0.054,
        0.066, 0.086, 0.033, 0.01 , 0.027, 0.001, 0.023, 0.   ],
       [0.075, 0.016, 0.02 , 0.04 , 0.12 , 0.023, 0.022, 0.069, 0.062,
        0.001, 0.007, 0.05 , 0.029, 0.062, 0.081, 0.016, 0.   , 0.062,
        0.072, 0.081, 0.028, 0.01 , 0.028, 0.001, 0.025, 0.001],
       [0.071, 0.017, 0.021, 0.042, 0.125, 0.025, 0.022, 0.072, 0.062,
        0.001, 0.008, 0.045, 0.021, 0.068, 0.072, 0.017, 0.001, 0.069,
        0.074, 0.09 , 0.027, 0.009, 0.026, 0.001, 0.015, 0.001],
       [0.078, 0.015, 0.021, 0.051, 0.118, 0.025, 0.02 , 0.07 , 0.065,
        0.002, 0.006, 0.04 , 0.022, 0.069, 0.069, 0.018, 0.001, 0.068,
        0.078, 0.084, 0.029, 0

The Hadamard product

In [111]:
hv_dist * hv_dist


array([[6.558e-03, 2.014e-04, 3.366e-04, 2.022e-03, 1.512e-02, 6.542e-04,
        3.997e-04, 6.347e-03, 3.667e-03, 6.645e-06, 4.906e-05, 1.614e-03,
        6.983e-04, 4.485e-03, 6.622e-03, 2.088e-04, 2.018e-07, 3.349e-03,
        4.285e-03, 7.395e-03, 8.538e-04, 9.252e-05, 6.182e-04, 8.979e-07,
        3.572e-04, 2.032e-07],
       [6.330e-03, 1.946e-04, 3.291e-04, 1.922e-03, 1.598e-02, 4.881e-04,
        4.296e-04, 5.409e-03, 3.706e-03, 8.036e-07, 5.893e-05, 1.605e-03,
        7.624e-04, 4.546e-03, 6.722e-03, 1.994e-04, 2.930e-07, 2.945e-03,
        4.394e-03, 7.326e-03, 1.061e-03, 1.031e-04, 7.542e-04, 5.476e-07,
        5.383e-04, 6.873e-08],
       [5.588e-03, 2.531e-04, 3.960e-04, 1.571e-03, 1.444e-02, 5.422e-04,
        4.946e-04, 4.769e-03, 3.842e-03, 3.667e-07, 5.441e-05, 2.479e-03,
        8.241e-04, 3.828e-03, 6.584e-03, 2.453e-04, 1.091e-07, 3.788e-03,
        5.192e-03, 6.548e-03, 8.053e-04, 9.873e-05, 7.928e-04, 1.094e-06,
        6.218e-04, 3.667e-07],
       [4.999e-03, 

## Matrix Products

### Matrix-Vector Multiplication

The product of a matrix  ${X}$ by a vector $\mathbf{y}$, ${X}\mathbf{y}$, is a sequence of dot products between the matrix rows and the vector resulting in a column vector:
$$
{X}\mathbf{y} = 
\begin{bmatrix*}
{X}_{1 .} \cdot \mathbf{y} \\
{X}_{2 .} \cdot \mathbf{y} \\
...\\
{X}_{n .} \cdot \mathbf{y} \\
\end{bmatrix*},
$$
where ${X}_{i .}$ denotes the $i^\text{th}$ row of matrix ${X}$. If ${X}$ consists of only one row, we have a matrix product of a row vector by a column vector, which is equivalent to a dot product:
$$
\mathbf{x} \cdot \mathbf{y} =
\begin{bmatrix*}
x_1,&x_2,& ...& x_n\\
\end{bmatrix*}
\begin{bmatrix*}
y_1\\
y_2\\ 
...\\
y_n\\
\end{bmatrix*}
= \sum_{i = 1}^n x_i y_i.
$$

In [112]:
hv_dist[0, :].reshape(1, -1) @ hv_dist[1, :]


array([0.066])

In [113]:
np.dot(hv_dist[0, :], hv_dist[1, :])


0.06581109734214702

In [114]:
hv_dist[0, :] @ hv_dist[1, :]


0.06581109734214702

### Document Cosines

We will now compute the cosine of all the pairs of vectors representing the works in the `hv_dist` matrix, _i.e._ the rows of the matrix. For this, we will first compute the dot products of all the pairs, $\mathbf{u} \cdot \mathbf{v}$, then the norms $||\mathbf{u}||$ and  $||\mathbf{v}||$, the products of the norms, $||\mathbf{u}|| \cdot||\mathbf{v}||$, and finally the cosines, $\displaystyle{\frac{\mathbf{u} \cdot \mathbf{v}}{||\mathbf{u}|| \cdot||\mathbf{v}||}}$.

The dot product, $\mathbf{u} \cdot \mathbf{v}$, of all the rows of a matrix $\mathbf{X}$ is simply $\mathbf{X} \mathbf{X}^\intercal$:

In [115]:
hv_dot = hv_dist @ hv_dist.T
hv_dot


array([[0.066, 0.066, 0.065, 0.066, 0.065],
       [0.066, 0.066, 0.065, 0.066, 0.065],
       [0.065, 0.065, 0.064, 0.065, 0.064],
       [0.066, 0.066, 0.065, 0.066, 0.065],
       [0.065, 0.065, 0.064, 0.065, 0.065]])

For the vector noms, $||\mathbf{u}||$ and  $||\mathbf{v}||$, we can use `np.linalg.norm()`. Here we will break down the computation with elementary operations. We will apply the Hadamard product to have the square of the coordinates, then sum along the rows, and finally extract the square root:

In [116]:
hv_norm = np.sqrt(np.sum(hv_dist * hv_dist, axis=1))
hv_norm


array([0.257, 0.257, 0.253, 0.257, 0.255])

We compute the product of the norms, $||\mathbf{u}|| \cdot||\mathbf{v}||$, as a matrix product of a column vector by a row vector as with:
$$
\begin{bmatrix*}
x_1\\
x_2\\
 ...\\
 x_n\\
\end{bmatrix*}
\begin{bmatrix*}
y_1, y_2, ..., y_n\\
\end{bmatrix*}
=
\begin{bmatrix*}
x_1 y_1& x_1 y_2&...&x_1 y_n\\
x_2 y_1& x_2 y_1&...&x_2 y_n\\
 ...\\
 x_ny_1& x_n y_2&...&x_n y_n \\
\end{bmatrix*}. 
$$


In [117]:
hv_norm_pairs = hv_norm.reshape(-1, 1) @ hv_norm.reshape(1, -1)
hv_norm_pairs


array([[0.066, 0.066, 0.065, 0.066, 0.065],
       [0.066, 0.066, 0.065, 0.066, 0.065],
       [0.065, 0.065, 0.064, 0.065, 0.064],
       [0.066, 0.066, 0.065, 0.066, 0.065],
       [0.065, 0.065, 0.064, 0.065, 0.065]])

We now nearly done with the cosine. We only need to divide the matrix elements by the norm products, $\displaystyle{\frac{\mathbf{u} \cdot \mathbf{v}}{||\mathbf{u}|| \cdot||\mathbf{v}||}}$

In [118]:
hv_cos = hv_dot / hv_norm_pairs
hv_cos


array([[1.   , 0.999, 0.997, 0.996, 0.995],
       [0.999, 1.   , 0.997, 0.995, 0.994],
       [0.997, 0.997, 1.   , 0.996, 0.995],
       [0.996, 0.995, 0.996, 1.   , 0.998],
       [0.995, 0.994, 0.995, 0.998, 1.   ]])

## Elementary Mathematical Background for Matrices

In [119]:
A = np.array([[1, 2],
              [3, 4]])
A @ np.array([5, 6])


array([17, 39])

### Matrices and Rotations

To finish this notebook, we will have a look at vector rotation. From algebra courses, we know that we can use a matrix to compute a rotation of angle $\theta$. For a two-dimensional vector, the rotation matrix is:
$$
{R}_{\theta} =
\begin{bmatrix*}
\cos \theta &-\sin \theta \\
\sin \theta & \cos \theta \\
\end{bmatrix*}.
$$

In [120]:
theta_45 = np.pi/4
rot_mat_45 = np.array([[np.cos(theta_45), -np.sin(theta_45)],
                       [np.sin(theta_45), np.cos(theta_45)]])
rot_mat_45


array([[ 0.707, -0.707],
       [ 0.707,  0.707]])

we rotate vector (1, 1) by this angle

In [121]:
rot_mat_45 @ np.array([1, 1])


array([1.110e-16, 1.414e+00])

The matrix of a sequence of rotations, for instance a rotation of $\pi/6$ followed by a rotation of $\pi/4$, is simply the matrix product of the individual rotations ${R}_{{\theta}_1} {R}_{{\theta}_2}  = {R}_{{\theta}_1 + {\theta}_2}$, here ${R}_{\pi/4} {R}_{\pi/6}  = {R}_{5\pi/12}$. 

In [122]:
theta_30 = np.pi/6
rot_mat_30 = np.array([[np.cos(theta_30), -np.sin(theta_30)],
                       [np.sin(theta_30), np.cos(theta_30)]])
rot_mat_30


array([[ 0.866, -0.5  ],
       [ 0.5  ,  0.866]])

rot_mat_30 @ rot_mat_45


In [123]:
rot_mat_45 @ rot_mat_30


array([[ 0.259, -0.966],
       [ 0.966,  0.259]])

In [124]:
np.arccos(0.25881905)


1.3089969339255036

In [125]:
np.pi/4 + np.pi/6


1.308996938995747

### Inverting a Matrix

In [126]:
np.linalg.inv(rot_mat_30)


array([[ 0.866,  0.5  ],
       [-0.5  ,  0.866]])

In [127]:
np.linalg.inv(rot_mat_30) @ rot_mat_30


array([[1.000e+00, 7.437e-18],
       [6.295e-17, 1.000e+00]])

In [128]:
torch.inverse(torch.from_numpy(rot_mat_30))


tensor([[ 0.8660,  0.5000],
        [-0.5000,  0.8660]], dtype=torch.float64)

In [129]:
torch.inverse(torch.from_numpy(rot_mat_30)) @ torch.from_numpy(rot_mat_30)


tensor([[ 1.0000e+00, -5.5511e-17],
        [ 5.5511e-17,  1.0000e+00]], dtype=torch.float64)

## Application to Neural Netwoks

### PyTorch

In [130]:
layer1 = torch.nn.Linear(3, 4, bias=False)


In [131]:
layer1.weight


Parameter containing:
tensor([[-0.4324,  0.0435,  0.1806],
        [-0.5352,  0.0966,  0.2330],
        [-0.2231,  0.5196, -0.0784],
        [-0.2372,  0.1172, -0.3739]], requires_grad=True)

In [132]:
x = torch.tensor([1.0, 2.0, 3.0])


In [133]:
layer1(x)


tensor([ 0.1963,  0.3571,  0.5810, -1.1245], grad_fn=<SqueezeBackward4>)

In [134]:
layer1.weight @ x


tensor([ 0.1963,  0.3571,  0.5810, -1.1245], grad_fn=<MvBackward0>)

Or see: https://pytorch.org/docs/stable/generated/torch.nn.Linear.html

In [135]:
x @ layer1.weight.T


tensor([ 0.1963,  0.3571,  0.5810, -1.1245], grad_fn=<SqueezeBackward4>)

### More Layers

In [136]:
layer1 = torch.nn.Linear(3, 4, bias=False)
layer2 = torch.nn.Linear(4, 2, bias=False)
layer3 = torch.nn.Linear(2, 1, bias=False)


In [137]:
(layer1.weight, layer2.weight, layer3.weight)


(Parameter containing:
 tensor([[ 0.5711, -0.2106,  0.5642],
         [-0.1257,  0.3728, -0.4489],
         [-0.1961, -0.0592, -0.0630],
         [-0.4868,  0.2738,  0.5165]], requires_grad=True),
 Parameter containing:
 tensor([[ 0.3872,  0.2588,  0.3612, -0.1009],
         [ 0.2428, -0.2557, -0.2185, -0.2425]], requires_grad=True),
 Parameter containing:
 tensor([[-0.7031, -0.4677]], requires_grad=True))

In [138]:
layer3(layer2(layer1(x)))


tensor([-0.2923], grad_fn=<SqueezeBackward4>)

In [139]:
x @ layer1.weight.T @ layer2.weight.T @ layer3.weight.T


tensor([-0.2923], grad_fn=<SqueezeBackward4>)

## Automatic Differentiation

A 3D curve:
$$
z = x^2 + xy + y^2
$$
Its gradient:
$$
\begin{array}{lcl}
\frac{\partial z}{\partial x} &=& 2x + y\\
\frac{\partial z}{\partial y} &=& x + 2y\\
\end{array}
$$

In [140]:
def f(x, y):
    return x**2 + x * y + y**2


In [141]:
xt = torch.tensor(3.0, requires_grad=True)
yt = torch.tensor(4.0, requires_grad=True)


In [142]:
zt = f(xt, yt)
zt


tensor(37., grad_fn=<AddBackward0>)

In [143]:
zt.backward()


In [144]:
zt


tensor(37., grad_fn=<AddBackward0>)

In [145]:
zt.grad_fn


In [146]:
zt.grad_fn.next_functions


((<AddBackward0 at 0x7f0b5c497100>, 0), (<PowBackward0 at 0x7f0b5c497c70>, 0))

In [147]:
zt.grad_fn.next_functions[0][0].next_functions


((<PowBackward0 at 0x7f0a8a58d8d0>, 0), (<MulBackward0 at 0x7f0b6f76f9a0>, 0))

In [148]:
zt.grad_fn.next_functions[0][0].next_functions[0][0].next_functions


((<AccumulateGrad at 0x7f0a88b5bb80>, 0),)

In [149]:
zt.grad_fn.next_functions[0][0].next_functions[0][0].next_functions[0][0].next_functions


()

In [150]:
xt.grad, yt.grad


(tensor(10.), tensor(11.))